In [48]:
import pandas as pd
import numpy as np
import glob
import datetime

## Load data

In [49]:
testing_centres_df = pd.read_csv("./data sources/assessment_centre_location.csv",
                                header =0,
                                usecols=[3,5,7,10,11,12,13, 15, 19,20,21,22,23,24,25,28,29])

mobility_trends_df = pd.read_csv("./data sources/2020_CA_Region_Mobility_Report.csv",
                                header=0,
                                usecols=[1,2,3,7,8,9,10,11,12,13]) 

all_weather_files = glob.glob("./data sources/ottawa_weather/*.csv") + glob.glob("./data sources/toronto_weather/*.csv")
weather_df = pd.concat((pd.read_csv(f, header=0, usecols=[0,1,2,4,13,19,21,23]) for f in all_weather_files))

phu_df = pd.read_csv("./data sources/confirmed_positive_cases_of_covid_19_in_ontario.csv", header =0, usecols=[12,13,14,15,16,17,18]).drop_duplicates()

START_DATE= datetime.datetime(2020,8,1)
END_DATE= datetime.datetime(2020,12,1)

In [50]:
def print_all_df_shapes():
    print(testing_centres_df.shape)
    print(mobility_trends_df.shape)
    print(weather_df.shape)
    print(phu_df.shape)
    
print_all_df_shapes()

(442, 17)
(84094, 10)
(732, 8)
(34, 7)


In [51]:
phu_df.head()

,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
0,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
11,Halton Region Health Department,1151 Bronte Road,Oakville,L6M 3Ll,www.halton.ca/For-Residents/Public-Health/,43.413997,-79.744796
19,Grey Bruce Health Unit,101 17th Street East,Owen Sound,N4K 0A5,www.publichealthgreybruce.on.ca/,44.576196,-80.940980
43,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
102,"Kingston, Frontenac and Lennox & Addington Pub...",221 Portsmouth Avenue,Kingston,K7M 1V5,www.kflaph.ca,44.227874,-76.525211


In [52]:
mobility_trends_df.head()

,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,Canada,NaN,NaN,2020-02-15,4.0,2.0,10.0,3.0,1.0,0.0
1,Canada,NaN,NaN,2020-02-16,13.0,8.0,41.0,4.0,0.0,-2.0
2,Canada,NaN,NaN,2020-02-17,-12.0,-15.0,63.0,-28.0,-52.0,11.0
3,Canada,NaN,NaN,2020-02-18,-1.0,4.0,6.0,-1.0,-1.0,1.0
4,Canada,NaN,NaN,2020-02-19,1.0,1.0,9.0,0.0,0.0,0.0


In [53]:
weather_df.head()

,Longitude (x),Latitude (y),Station Name,Date/Time,Mean Temp (°C),Total Rain (mm),Total Snow (cm),Total Precip (mm)
0,-75.72,45.38,OTTAWA CDA,2020-01-01,-0.3,1.0,0.0,1.0
1,-75.72,45.38,OTTAWA CDA,2020-01-02,0.3,0.0,0.0,0.0
2,-75.72,45.38,OTTAWA CDA,2020-01-03,3.8,0.0,0.0,0.0
3,-75.72,45.38,OTTAWA CDA,2020-01-04,0.0,0.0,3.0,1.8
4,-75.72,45.38,OTTAWA CDA,2020-01-05,-6.5,0.0,1.0,0.8


## Process data

### Testing Centres Data

In [54]:
#filter by row
testing_centres_df = testing_centres_df.loc[testing_centres_df['city'].isin(["Ottawa","Toronto","Toronto "])]

#testing_centres_df.rename({"sub_region_1":"province"})

#get total hours open
def hours_open_row(row, col1, col2):
    tot_hours = 0
    tot_days = 0
    for i in range(col1,col2):
        if isinstance(row[i],str):
            nums = row[i].split("-")
            nums[0] = int(nums[0][0:2])
            nums[1] = int(nums[1][0:2])
            tot_hours+= nums[1] - nums[0]
            tot_days+=1
    if tot_days ==0:
        return 0
    return tot_hours

def to_bool(row, column):
    if not(isinstance(row[column], str)):
        return None
    string = row[column].strip().lower()
    if string == "no":
        return False
    elif string == "yes":
        return True
    return None

# testing_centres_df['hours_open_weekday'] = testing_centres_df.apply (lambda row: hours_open_row(row,8,13), axis=1)
# testing_centres_df['hours_open_weekend'] = testing_centres_df.apply (lambda row: hours_open_row(row,13,15), axis=1)
# testing_centres_df.drop(["monday","tuesday","wednesday","thursday","friday","saturday","sunday"], axis=1, inplace=True)

# #Change abreviation for province to full name
# testing_centres_df["province"] = testing_centres_df["province"].map({"ON":"Ontario"})

# #change values to boolean for walkins and drive through columns
# testing_centres_df['walk_ins'] = testing_centres_df.apply(lambda row: to_bool(row,"walk_ins"), axis=1).astype(bool)
# testing_centres_df['drive_through'] = testing_centres_df.apply(lambda row: to_bool(row,"drive_through"), axis=1).astype(bool)

# #create surrogte keys
# testing_centres_df.insert(0, "testing_centre_key",  np.arange(len(testing_centres_df)))

# #change column types to str
# testing_centres_df = testing_centres_df.astype(dtype= {"location_name":"string","operated_by":"string","postal_code":"string",
#                                                       "province":"string","PHU":"string"})

# #print df
# print(testing_centres_df.shape)
# print(testing_centres_df.dtypes)
# testing_centres_df.head()

In [55]:
#PHU DATA
phu_df=phu_df.loc[phu_df['Reporting_PHU_City'].isin(["Ottawa","Toronto","Mississauga","Oakville", "Newmarket","Whitby"])]
phu_df.columns = ["phu_name","address","city","postal_code","url","latitude","longitude"]
phu_df.insert(0, "phu_key",  np.arange(len(phu_df)))
phu_df.insert(1,"province", "Ontario")
phu_df['municipality']= phu_df['city'].map({'Mississauga':'Peel','Oakville':'Halton','Newmarket':'York','Whitby':'Durham','Toronto':'Toronto','Ottawa':'Ottawa'})
phu_df.head(6)

,phu_key,province,phu_name,address,city,postal_code,url,latitude,longitude,municipality
0,0,Ontario,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893,Peel
11,1,Ontario,Halton Region Health Department,1151 Bronte Road,Oakville,L6M 3Ll,www.halton.ca/For-Residents/Public-Health/,43.413997,-79.744796,Halton
43,2,Ontario,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358,Toronto
1600,3,Ontario,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912,Ottawa
1605,4,Ontario,York Region Public Health Services,17250 Yonge Street,Newmarket,L3Y 6Z1,www.york.ca/wps/portal/yorkhome/health/,44.048023,-79.480239,York
1679,5,Ontario,Durham Region Health Department,605 Rossland Road East,Whitby,L1N 0B2,www.durham.ca/en/health-and-wellness/health-an...,43.898605,-78.940341,Durham


### Mobility Data

In [56]:
# Filter by rows
cities_mobility = ["Ottawa Division","Toronto Division", "Regional Municipality of Peel", "Regional Municipality of York","Regional Municipality of Durham","Regional Municipality of Halton"]
mobility_trends_df= mobility_trends_df.loc[mobility_trends_df['sub_region_2'].isin(cities_mobility)]

# filter rows by date
mobility_trends_df["date"]= pd.to_datetime(mobility_trends_df["date"])
mobility_trends_df= mobility_trends_df[(mobility_trends_df['date']>=START_DATE) & (mobility_trends_df['date']<END_DATE) ]

# only keep city name in sub_region_2 column in mobility df
def mobility_city_rename(row):
    if row["sub_region_2"][0:8] == "Regional":
        return row["sub_region_2"].split(" ")[-1]
    return row["sub_region_2"].split(" ")[0]

mobility_trends_df['sub_region_2'] =  mobility_trends_df.apply (lambda row: mobility_city_rename(row), axis=1)

#rename columns
mobility_trends_df.columns = ["country","province","municipality","date","retail_and_recreation",
                             "grocery_and_pharmacy", "parks","transit_stations",
                             "workplaces", "residential"]

#change ojbect type
mobility_trends_df = mobility_trends_df.astype(dtype= {"country":"string","province":"string","municipality":"string"})

#create surrogte keys
mobility_trends_df.insert(0, "mobility_key",  np.arange(len(mobility_trends_df)))

#print df
print(mobility_trends_df.shape)
print(mobility_trends_df.dtypes)
mobility_trends_df.head(5)

(732, 11)
mobility_key                      int32
country                          string
province                         string
municipality                     string
date                     datetime64[ns]
retail_and_recreation           float64
grocery_and_pharmacy            float64
parks                           float64
transit_stations                float64
workplaces                      float64
residential                     float64
dtype: object


,mobility_key,country,province,municipality,date,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
42572,0,Canada,Ontario,Ottawa,2020-08-01,-26.0,-9.0,181.0,-32.0,-2.0,0.0
42573,1,Canada,Ontario,Ottawa,2020-08-02,-25.0,-11.0,2.0,-46.0,-7.0,3.0
42574,2,Canada,Ontario,Ottawa,2020-08-03,-35.0,-24.0,137.0,-67.0,-77.0,17.0
42575,3,Canada,Ontario,Ottawa,2020-08-04,-27.0,-8.0,-21.0,-62.0,-61.0,19.0
42576,4,Canada,Ontario,Ottawa,2020-08-05,-20.0,1.0,59.0,-60.0,-58.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...
45081,195,Canada,Ontario,Durham,2020-10-13,-13.0,5.0,35.0,-53.0,-39.0,12.0
45082,196,Canada,Ontario,Durham,2020-10-14,-13.0,1.0,32.0,-57.0,-35.0,13.0
45083,197,Canada,Ontario,Durham,2020-10-15,-18.0,-2.0,-29.0,-58.0,-37.0,15.0
45084,198,Canada,Ontario,Durham,2020-10-16,-15.0,0.0,17.0,-51.0,-33.0,12.0


### Weather Data

In [57]:
# filter rows by date
weather_df["Date/Time"]= pd.to_datetime(weather_df["Date/Time"])
weather_df= weather_df[(weather_df['Date/Time']>=START_DATE) & (weather_df['Date/Time']<END_DATE) ]

#add prec_type column from  total rain and total snow column
def rain_col(row):
    if row["Total Rain (mm)"] > 0.0:
        return True
    return False
    
def snow_col(row):
    if row["Total Snow (cm)"] > 0.0:
        return True
    return False
    
weather_df['snow'] =  weather_df.apply (lambda row: snow_col(row), axis=1)
weather_df['rain'] =  weather_df.apply (lambda row: rain_col(row), axis=1)
weather_df.drop(["Total Rain (mm)", "Total Snow (cm)"], axis=1, inplace=True)

#rename columns
weather_df.columns = ["longitude","latitude","station","date","mean_temp","total_precipitation_mm","snow","rain"]

#change ojbect type
weather_df = weather_df.astype(dtype= {"station":"string"})

#create surrogte keys
weather_df.insert(0, "weather_key",  np.arange(len(weather_df)))

print(weather_df.dtypes)
print(weather_df.shape)
weather_df.head()

weather_key                        int32
longitude                        float64
latitude                         float64
station                           string
date                      datetime64[ns]
mean_temp                        float64
total_precipitation_mm           float64
snow                                bool
rain                                bool
dtype: object
(244, 9)


,weather_key,longitude,latitude,station,date,mean_temp,total_precipitation_mm,snow,rain
213,0,-75.72,45.38,OTTAWA CDA,2020-08-01,22.8,0.0,False,False
214,1,-75.72,45.38,OTTAWA CDA,2020-08-02,21.5,36.0,False,True
215,2,-75.72,45.38,OTTAWA CDA,2020-08-03,20.8,2.0,False,True
216,3,-75.72,45.38,OTTAWA CDA,2020-08-04,19.0,22.2,False,True
217,4,-75.72,45.38,OTTAWA CDA,2020-08-05,20.3,1.0,False,True


## Export Data 

### to be Used to Get Foreign Keys for Fact Table

In [58]:
#Export dfs so they can be merged with fact table to get foreign keys

#weather_df.to_csv("./processed_data/weather.csv")
mobility_trends_df.to_csv("./processed_data/mobility_trends.csv")
phu_df.to_csv("./processed_data/phu.csv")

# fact_table = pd.merge(fact_table,weather_df, left_on='date_reported', right_on="date")

In [59]:
#Remove date column
weather_df.drop(["date"], axis=1, inplace=True)
mobility_trends_df.drop(["date"], axis=1, inplace=True)

## Database connection

#### Push data to postgres server

In [60]:
from sqlalchemy import create_engine, text
import psycopg2 
import io
from config import PASSWORD, HOST, USER

engine = create_engine('postgresql://'+USER+':'+PASSWORD+HOST+'/group_12')

#create tables from SQL file called schema.txt
def create_tables(engine):
    sql_file = open("./schema.sql")
    escaped_sql = text(sql_file.read())
    engine.execute(escaped_sql)
    
#load data from datafram to database
def push_df_to_db(engine, df, table_name):
    try:
        conn = engine.raw_connection()
        cur = conn.cursor()
        output = io.StringIO()
        df.to_csv(output, sep='\t', header=False, index=False)
        output.seek(0)
        contents = output.getvalue()
        cur.copy_from(output, table_name, null="") # null values become ''
        conn.commit()
    except Exception as e:
        print(e)
    finally:
        conn.close()


In [63]:
# execute once
create_tables(engine)

In [64]:
# load data for each table, execute once
push_df_to_db(engine, mobility_trends_df, "mobility_trends")
#push_df_to_db(engine, weather_df, "weather")
push_df_to_db(engine, phu_df, "phu")
